In [21]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn.modules.pooling import MaxPool2d
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-saqm0lz_
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-saqm0lz_
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [22]:
# FC to MPS

# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
nepochs_teacher_HP = 25
teacher_loss_HP = nn.CrossEntropyLoss()
teacher_lr_HP = 1e-2
teacher_reg_HP = 0.01
teacher_hidden_size_HP = 70
# Student hyperparameters
# MPS parameters
bond_dim_HP = 40
adaptive_mode_HP = False
periodic_bc_HP = False

# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 25 
student_lr_HP = 1e-4
student_reg_HP = 0.01
student_loss_HP = nn.KLDivLoss(reduction = "batchmean", log_target = True)

# Gaussian parameters
gauss_epochs_HP = 0 # number of epochs with added gaussian noise
gn_var_HP = 0.3 #added gaussian noise variance
gn_mean_HP = 0 #added gaussian noise mean
nepochs_student_HP = 25 + gauss_epochs_HP


# Premilinaries: Importing the data and utils subroutines

In [23]:
# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [24]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

In [25]:
# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, teacher_hidden_size_HP)
        self.lin2 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin3 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin4 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin5 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin6 = nn.Linear(teacher_hidden_size_HP, 10)

    def middleforward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        return y

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

teacher = torch.load('/content/honor_project/src/fc_0.541.pt')


# Training the student model

In [26]:
def train_a_student():
  '''
  Trains a student model.
  '''
  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device)
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  # Training loop 
  for epoch in range(nepochs_student_HP):
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          # Add Gaussian noise for the gaussian epochs
          if (epoch >= nepochs_student_HP - gauss_epochs_HP):
            x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

          student_output = LogSoftmax( student(x_mb) )
          teacher_output = LogSoftmax( teacher(x_mb) )

          # Backpropagation
          loss = student_loss_HP(student_output, teacher_output)
          loss.backward()
          student_optimizer.step()
          student_optimizer.zero_grad()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)

# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))



 Training and testing of model  1  in progress...
Train loss:  [0.1    0.1    0.1939 0.396  0.3932 0.4715 0.4974 0.536  0.5255 0.5394
 0.5588 0.5526 0.5745 0.5709 0.5784 0.581  0.579  0.5823 0.5911 0.5824
 0.5844 0.6003 0.5823 0.5865 0.5913]
Test loss:  [0.1    0.1    0.1846 0.3839 0.3929 0.4484 0.482  0.521  0.5187 0.5368
 0.5423 0.5302 0.5505 0.5549 0.5519 0.5394 0.5469 0.5489 0.554  0.5573
 0.5638 0.5755 0.5533 0.5574 0.5662]

 Training and testing of model  2  in progress...
Train loss:  [0.1    0.1    0.1    0.1    0.1778 0.1    0.1    0.1    0.1    0.1
 0.2175 0.2862 0.4188 0.468  0.5136 0.5072 0.5445 0.5611 0.5657 0.5732
 0.5646 0.5791 0.5784 0.5865 0.5809]
Test loss:  [0.1    0.1    0.1    0.1    0.1865 0.1    0.1    0.1    0.1    0.1
 0.2182 0.2799 0.4002 0.4698 0.4938 0.513  0.5312 0.5362 0.5472 0.5511
 0.5514 0.5467 0.5527 0.5497 0.5461]

 Training and testing of model  3  in progress...
Train loss:  [0.1    0.1    0.1    0.1    0.2336 0.3104 0.3583 0.4675 0.5234 0.5264
 0.

In [27]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_fc_to_mps_trainloss_' + str(bond_dim_HP) + 'bd', global_stud_train_loss)
np.save('20x_fc_to_mps_testloss_' + str(bond_dim_HP) + 'bd', global_stud_test_loss)



Final results
Train loss:  [[0.1    0.1    0.1939 0.396  0.3932 0.4715 0.4974 0.536  0.5255 0.5394
  0.5588 0.5526 0.5745 0.5709 0.5784 0.581  0.579  0.5823 0.5911 0.5824
  0.5844 0.6003 0.5823 0.5865 0.5913]
 [0.1    0.1    0.1    0.1    0.1778 0.1    0.1    0.1    0.1    0.1
  0.2175 0.2862 0.4188 0.468  0.5136 0.5072 0.5445 0.5611 0.5657 0.5732
  0.5646 0.5791 0.5784 0.5865 0.5809]
 [0.1    0.1    0.1    0.1    0.2336 0.3104 0.3583 0.4675 0.5234 0.5264
  0.5387 0.5577 0.5578 0.5712 0.5777 0.57   0.5793 0.5797 0.5815 0.5912
  0.5838 0.5832 0.5814 0.5888 0.5918]
 [0.1    0.1    0.1    0.1617 0.3829 0.4654 0.4881 0.4885 0.5278 0.5254
  0.5625 0.5628 0.57   0.5721 0.5658 0.5792 0.5783 0.5764 0.5825 0.5851
  0.5875 0.585  0.5881 0.5894 0.5836]
 [0.1    0.1    0.1018 0.2304 0.3317 0.4192 0.4849 0.5344 0.5342 0.5489
  0.5635 0.5717 0.5678 0.5814 0.581  0.58   0.5577 0.5832 0.5859 0.5857
  0.5882 0.5918 0.5882 0.5904 0.5897]
 [0.1    0.1    0.1    0.1    0.1    0.1    0.1814 0.3356 0.4579 0